In [46]:
import network_utils as nu
import pandas as pd
import binary_tree as bt

rules = nu.read_rules('rule_list.txt')

pd.DataFrame([vars(r) for r in rules])

,src_sub,dst_sub,protocol,src_port,dst_port,action,src_ip,src_netmask,src_sub_binary,dst_ip,dst_netmask,dst_sub_binary
0,192.168.1.1/24,17.42.1/24,5,*,*,ALLOW,192.168.1.1,24,110000001010100000000001,17.42.1,24.0,000100010010101000000001
1,17.10.0.1/16,*,*,*,*,DENY,17.10.0.1,16,0001000100001010,NaN,NaN,NaN
2,17/8,*,*,*,*,DENY,17,8,00010001,NaN,NaN,NaN
3,252.31/11,131.16/14,5,21,80,DENY,252.31,11,11111100000,131.16,14.0,10000011000100
4,255.255.255.255/32,255.255.255.255/32,*,6,1,DROP,255.255.255.255,32,11111111111111111111111111111111,255.255.255.255,32.0,11111111111111111111111111111111


In [47]:
# Generate Random packets
f = open("random_packets.txt", "w")
for i in range(0,10000):
    f.write(nu.packet_generator()+"\n")
    
f.close()

In [48]:
packets = nu.read_packets("incoming_packets.txt")
packets = nu.read_packets("random_packets.txt")

pd.DataFrame([vars(r) for r in packets])  

,src_ip,dst_ip,protocol,src_port,dst_port,src_binary,dst_binary
0,17.25.12.170,17.66.96.247,146,29122,62724,00010001000110010000110010101010,00010001010000100110000011110111
1,17.161.160.161,17.179.154.143,149,41271,27820,00010001101000011010000010100001,00010001101100111001101010001111
2,17.99.182.192,17.167.198.123,146,17482,12764,00010001011000111011011011000000,00010001101001111100011001111011
3,17.245.163.11,17.228.252.156,84,49485,13107,00010001111101011010001100001011,00010001111001001111110010011100
4,17.235.198.111,17.9.236.59,255,27862,3003,00010001111010111100011001101111,00010001000010011110110000111011
...,...,...,...,...,...,...,...
9995,17.101.135.125,17.227.92.179,77,55794,47514,00010001011001011000011101111101,00010001111000110101110010110011
9996,17.80.93.108,17.74.99.160,236,59415,27406,00010001010100000101110101101100,00010001010010100110001110100000
9997,17.64.224.179,17.248.171.47,205,19486,38218,00010001010000001110000010110011,00010001111110001010101100101111
9998,17.195.129.115,17.192.164.124,31,1189,42243,00010001110000111000000101110011,00010001110000001010010001111100


In [49]:
root = bt.Node()    

r1 = ["1100", "0101", "0000", "1011", "1100", "1101","1100",None,None]
r2 = ["1001", "1101", "1010", "1001", "1000", None, "1000",None, "1010"]

for i in range(0 , len(r1)):
    bt.add_src_nodes(root, r1[i], 0, r2[i], i)
    
bt.show(root)

In [50]:
p1 = ["1100", "011101", "0000", "1010", "11000", "1101"]
p2 = ["10010", "11010", "10100", "1000", "11010", "0000"]


for i in range(0, len(p1)):
    actions = []
    bt.match(root, p1[i], 0, p2[i], 0, actions)
    print(actions)

[7, 0]
[7]
[7, 8, 2]
[7]
[7]
[7, 5]


In [51]:
root = bt.Node()    

src_sub_binaries = [x.src_sub_binary for x in rules]
dst_sub_binaries = [x.dst_sub_binary for x in rules]

for i in range(0, len(src_sub_binaries)):
    bt.add_src_nodes(root, src_sub_binaries[i], 0, dst_sub_binaries[i],i)
    
bt.show(root)

True
False
True
True
False
True
True


In [58]:
actions=[]

for packet in packets:
    candidate_actions = []
    bt.match(root, packet.src_binary, 0, packet.dst_binary, 0, candidate_actions)
    
    final_actions = []
    
    for i in candidate_actions:
        if rules[i].protocol != '*' and rules[i].protocol != packet.protocol:
            continue
        if not nu.is_in_port_range(rules[i].src_port, packet.src_port):
            continue
        if not nu.is_in_port_range(rules[i].dst_port, packet.dst_port):
            continue
        final_actions.append(i)
        
    actions.append(rules[sorted(final_actions)[0]].action)
 
print(actions)

['DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY', 'DENY',